In [1]:
import pyspark
from pyspark import SQLContext, SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf()
conf.setAppName("project1")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)


In [2]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
spark = SparkSession.builder.appName("project1").getOrCreate()

# Preprocessing

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType

schema = StructType([
    StructField("marketplace", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("review_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("product_parent", StringType(), True),
    StructField("label", StringType(), True), # "label" replaces "product_title"
    StructField("product_category", StringType(), True),
    StructField("star_rating", IntegerType(), True),
    StructField("helpful_votes", IntegerType(), True),
    StructField("total_votes", IntegerType(), True),
    StructField("vine", StringType(), True),
    StructField("verified_purchase", StringType(), True),
    StructField("review_headline", StringType(), True),
    StructField("review_body", StringType(), True),
    StructField("review_date", StringType(), True)])

df_grocery = spark.read.csv('amazon_reviews_us_Grocery_v1_00.tsv',sep="\t", header=True, schema=schema)
df_electronics = spark.read.csv('amazon_reviews_us_Electronics_v1_00.tsv',sep="\t", header=True, schema=schema)
df_videogames = spark.read.csv('amazon_reviews_us_Video_Games_v1_00.tsv',sep="\t", header=True, schema=schema)
df_toys = spark.read.csv('amazon_reviews_us_Toys_v1_00.tsv',sep="\t", header=True, schema=schema)

In [4]:
df_grocery.na.drop()
df_electronics.na.drop()
df_videogames.na.drop()
df_toys.na.drop()

DataFrame[marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, label: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [5]:
df_grocery.count() # number of entries

2402458

In [6]:
df_electronics.count() # number of entries

3093869

In [7]:
df_videogames.count() # number of entries

1785997

In [8]:
df_toys.count() # number of entries

4864249

In [9]:
df_grocery.show(5) # show first 10 entries

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|               label|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [10]:
df_electronics.show(5) # show first 10 entries

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|               label|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [11]:
df_videogames.show(5) # show first 10 entries

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|               label|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [12]:
df_toys.show(5) # show first 10 entries

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|               label|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [13]:
df1 = df_grocery.drop('marketplace','customer_id','review_id','product_id','product_parent','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_date')
df2 = df_electronics.drop('marketplace','customer_id','review_id','product_id','product_parent','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_date')
df3 = df_videogames.drop('marketplace','customer_id','review_id','product_id','product_parent','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_date')
df4 = df_toys.drop('marketplace','customer_id','review_id','product_id','product_parent','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_date')

In [14]:
# make sure values are nullable

In [15]:
df1.printSchema()

root
 |-- label: string (nullable = true)
 |-- review_body: string (nullable = true)



In [16]:
df2.printSchema()

root
 |-- label: string (nullable = true)
 |-- review_body: string (nullable = true)



In [17]:
df3.printSchema()

root
 |-- label: string (nullable = true)
 |-- review_body: string (nullable = true)



In [18]:
df4.printSchema()

root
 |-- label: string (nullable = true)
 |-- review_body: string (nullable = true)



In [33]:
df1.head()

Row(label='The Cravings Place Chocolate Chunk Cookie Mix, 23-Ounce Bags (Pack of 6)', review_body="As a family allergic to wheat, dairy, eggs, nuts, and several other things, we love the entire Cravings Place line of products as it allows us to bake treats with minimal effort and ingredients. Most allergy-free and gluten-free mixes usually just omit one or two allergens at most, so it's great to see a mix created without many of the most common allergens. (Note these still have soy and corn). We consume these on a regular basis and have been doing so for years.")

In [19]:
df1.count()

2402458

In [20]:
df2.count()

3093869

In [21]:
df3.count()

1785997

In [22]:
# sort by labels, top 20

In [23]:
from pyspark.sql.functions import col
df1.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
|               label|count|
+--------------------+-----+
|San Francisco Bay...|17032|
|Viva Naturals Org...|10068|
|Nutiva Organic Vi...| 5798|
| Davidson's Tea Bulk| 5716|
|Grove Square Capp...| 5146|
|Keurig Green Moun...| 4923|
|Amazing Grass Gre...| 4179|
|Surge Citrus Flav...| 3902|
|Brooklyn Beans Si...| 3853|
|Keurig, The Origi...| 3476|
|Ekobrew Coffee Re...| 3410|
|  Senseo Coffee Pods| 3231|
|Grove Square Hot ...| 3144|
|Twinings Earl Gre...| 2972|
|KIND PLUS Gluten ...| 2861|
|Celestial Seasoni...| 2837|
|Green Mountain Co...| 2462|
|            Twinings| 2393|
|Vita Coco Coconut...| 2343|
|Timothy's World C...| 2307|
+--------------------+-----+
only showing top 20 rows



In [24]:
from pyspark.sql.functions import col
df2.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
|               label|count|
+--------------------+-----+
|Panasonic ErgoFit...|24833|
|AmazonBasics High...|16163|
|Mediabridge ULTRA...|15675|
|Clip Plus 4 GB MP...|11779|
|High Speed HDMI C...|11177|
|AmazonBasics High...|10740|
|VideoSecu ML531BE...|10216|
|CABTE High speed ...| 9923|
|Cheetah APTMM2B T...| 9365|
|Sennheiser On-Ear...| 9104|
|Bluetooth Speaker...| 9005|
|     HDMI-High-Speed| 8168|
|Sanyo NEW 1500 en...| 7554|
|MEElectronics Spo...| 7248|
|Sony MDRZX100 Hea...| 7235|
|Mohu Leaf 30 TV A...| 6539|
|AmazonBasics Ultr...| 6454|
|Bose SoundLink Mi...| 6075|
|Belkin 6-Outlet H...| 5858|
|Mediabridge 3.5mm...| 5823|
+--------------------+-----+
only showing top 20 rows



In [25]:
from pyspark.sql.functions import col
df3.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
|               label|count|
+--------------------+-----+
|PlayStation 4 500...|10361|
|  Grand Theft Auto V| 8715|
|Call of Duty: Ghosts| 7810|
|       Battlefield 4| 4809|
|  Assassin's Creed 4| 4722|
|      The Last of Us| 4598|
|Elder Scrolls V: ...| 4537|
|             Destiny| 4410|
| Diablo III - PC/Mac| 4390|
|Call of Duty: Bla...| 4373|
|SimCity - Limited...| 3972|
|       Battlefield 3| 3953|
|      Rocksmith 2014| 3905|
|Call of Duty: Adv...| 3798|
|               Spore| 3591|
|     Nintendo Amiibo| 3445|
|Assassin's Creed III| 3405|
|E-3lue Cobra EMS1...| 3404|
|Microsoft Xbox360...| 3400|
|Minecraft - Xbox 360| 3368|
+--------------------+-----+
only showing top 20 rows



In [26]:
from pyspark.sql.functions import col
df4.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
|               label|count|
+--------------------+-----+
|Cards Against Hum...|24288|
|      Melissa & Doug|11657|
|Cards Against Hum...| 6060|
|Syma S107/S107G  ...| 5848|
|VTech Sit-to-Stan...| 5040|
|Cards Against Hum...| 3963|
|Syma S107/S107G R...| 3647|
|Snap Circuits Jr....| 2972|
|            Spot It!| 2919|
|The Original Stom...| 2886|
|Cards Against Hum...| 2741|
|Fisher-Price Ocea...| 2644|
|Accoutrements Hor...| 2464|
|UDI U818A 2.4GHz ...| 2457|
|Mega Bloks 80-Pie...| 2371|
|Disney Frozen Spa...| 2332|
|Cards Against Hum...| 2298|
|Rainbow Loom Craf...| 2116|
|      Ticket To Ride| 2070|
|The Settlers of C...| 2038|
+--------------------+-----+
only showing top 20 rows



# Text Processing

## 1 (a). Model Pipeline

In [27]:
# Tokenization
#regexTokenizer = RegexTokenizer(inputCol="review_body", outputCol="words", pattern="[^A-Za-z]+", toLowercase=True)
#tokenized_data = regexTokenizer.transform(df)

from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="review_body", outputCol="words", pattern="[^A-Za-z]+", toLowercase=True)

# stop words
add_stopwords = ["http","https","amp","rt","t","c","the"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

## 2 (b). StringIndexer

In [29]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "label", outputCol = "label_final")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# Fit the pipeline to training documents.
pipelineFit1 = pipeline.fit(df1)
pipelineFit2 = pipeline.fit(df2)
pipelineFit3 = pipeline.fit(df3)
pipelineFit4 = pipeline.fit(df4)

dataset1 = pipelineFit1.transform(df1)
dataset1.show(5)
dataset2 = pipelineFit2.transform(df2)
dataset2.show(5)
dataset3 = pipelineFit3.transform(df3)
dataset3.show(5)
dataset4 = pipelineFit4.transform(df4)
dataset4.show(5)

Py4JJavaError: An error occurred while calling o150.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 28.0 failed 1 times, most recent failure: Lost task 0.0 in stage 28.0 (TID 933, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$2: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:439)
	at org.apache.s

# Partition Training & Test Sets

In [ ]:
# set seed for reproducibility
(trainingData1, testData1) = dataset1.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset 1 Count: " + str(trainingData1.count()))
print("Test Dataset Count: " + str(testData1.count()))

(trainingData2, testData2) = dataset2.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset 2 Count: " + str(trainingData2.count()))
print("Test Dataset Count: " + str(testData2.count()))

(trainingData3, testData3) = dataset3.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset 3 Count: " + str(trainingData3.count()))
print("Test Dataset Count: " + str(testData3.count()))

(trainingData4, testData4) = dataset3.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset 3 Count: " + str(trainingData3.count()))
print("Test Dataset Count: " + str(testData3.count()))

# Model Training & Evaluation

In [ ]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData1)
predictions = lrModel.transform(testData1)
predictions.filter(predictions['prediction'] == 0) \
    .select("Descript","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)
    
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData2)
predictions = lrModel.transform(testData2)
predictions.filter(predictions['prediction'] == 0) \
    .select("Descript","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)
    
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData3)
predictions = lrModel.transform(testData3)
predictions.filter(predictions['prediction'] == 0) \
    .select("Descript","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)
    
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData4)
predictions = lrModel.transform(testData4)
predictions.filter(predictions['prediction'] == 0) \
    .select("Descript","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

# Clustering

In [30]:
# Tokenization
regexTokenizer = RegexTokenizer(inputCol="review_body", outputCol="words", pattern="[^A-Za-z]+", toLowercase=True)
tokenized_data = regexTokenizer.transform(df1)

In [31]:
# Stop Word Removal
from pyspark.ml.feature import StopWordsRemover
stopWordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filtered_data = stopWordsRemover.transform(tokenized_data)

In [32]:
# Vectorization
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=20)
featurizedData = hashingTF.transform(filtered_data)

idf= IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(featurizedData)
featurized_data = idfModel.transform(featurizedData)

Py4JJavaError: An error occurred while calling o375.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 30.0 failed 1 times, most recent failure: Lost task 0.0 in stage 30.0 (TID 935, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$2: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at scala.collectio

In [ ]:
# Featurized Data
from pyspark.sql.functions import rand
# Show 10 random entries with label and features
featurized_data.select("label", "features").orderBy(rand()).limit(10).collect()

In [ ]:
dataset = featurized_data.select("features")

# (a) K-Means

In [ ]:
from pyspark.ml.clustering import KMeans

# Train a k-means model
kmeans = KMeans().setK(3).setSeed(1)
model = kmeans.fit(dataset)

# Evaluate clustering by computing Within Set Sum of Squared Errors
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Show the result
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

In [ ]:
# Predictions
predictions_kmeans = model.transform(dataset)
predictions_kmeans.show(1)

## (b) Latent Dirichlet Allocation (LDA)

In [ ]:
from pyspark.ml.clustering import LDA

# Trains a LDA model
lda = LDA(k=3, maxIter=10)
model = lda.fit(dataset)

ll = model.logLikelihood(dataset)
lp = model.logPerplexity(dataset)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

# Describe topics
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

In [ ]:
# Predictions
predictions_lda = model.transform(dataset)
predictions_lda.show(1)

## (c) Gaussian Mixture Model (GMM)

In [ ]:
from pyspark.ml.clustering import GaussianMixture

gmm = GaussianMixture().setK(3).setSeed(538009335)
model = gmm.fit(dataset)

print("Gaussians shown as a DataFrame: ")
model.gaussiansDF.show(truncate=False)

In [ ]:
# Predictions
predictions_gmm = model.transform(dataset)
predictions_gmm.show(3)

In [ ]:
filtered_data.head()

In [ ]:
# Vectorization
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=10)
featurizedData = hashingTF.transform(filtered_data)

#idf= IDF(inputCol="raw_features", outputCol="features")
#idfModel = idf.fit(featurizedData)
#featurized_data = idfModel.transform(featurizedData)
featurized_data = featurizedData

In [ ]:
featurized_data.head()

In [ ]:
df_final = featurized_data.select("label", "raw_features")

In [ ]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
(train, test) = df_final.randomSplit([0.8, 0.2])


In [ ]:
train1 = sc.parallelize(train)

In [ ]:
# instantiate the base classifier.
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)


In [ ]:

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr)




In [ ]:
# train the multiclass model.
ovrModel = ovr.fit(train)



In [ ]:
# score the model on test data.
predictions = ovrModel.transform(test)

# obtain evaluator.
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

# compute the classification error on test data.
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

In [ ]:
# Featurized Data
from pyspark.sql.functions import rand
from pyspark.sql.types import ArrayType, DoubleType

vector_udf = udf(lambda vector: vector.toArray().tolist(), ArrayType(DoubleType()))
featurized_data2 = featurized_data.withColumn("feature_array", vector_udf(featurized_data.features))

output = featurized_data2.select(["id"] + [col("feature_array")[i] for i in range(20)] + ["label"])
output.repartition(1).write.save("/home/data.csv"
    ,format='csv'
    ,mode='overwrite'
)

In [ ]:
# Show 10 random entries with label and features
df_final = featurized_data.select("label", "features") #.orderBy(rand()).limit(10).collect()

In [ ]:
df_final.head()

In [ ]:
from  pyspark.mllib.linalg import SparseVector, DenseVector
import pandas as pd
import numpy as np

sparseVec = featurized_data.select("features")
#features = featurized_data.select("features").apply(lambda x : np.array(SparseVec.toArray())).as_matrix().reshape(-1,1)
def dense_to_array(v):
    new_array = list([float(x) for x in v])
    return new_array
#test = dense_to_array(sparseVec)
# SparseVec.head()
# features = pd.DataFrame(SparseVec.toArray())

In [ ]:
featurized_data.write.csv('data.csv')

In [ ]:
dataset.head()
type(dataset)

In [ ]:
type(df)

In [ ]:
dataset.write.csv('data.csv')